In [ ]:
import pandas as pd
import numpy as np
from sys import exit
import plotnine as p9


In [2]:
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

print(xmas.columns)

Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location'], dtype='object')


In [3]:
print(xmas)

    Day Day.in.Words  Gift.Item        Verb Adjective        Location
0     1        first  partridge         NaN       NaN  in a pear tree
1     2       second       dove         NaN    turtle             NaN
2     3        third        hen         NaN    french             NaN
3     4       fourth       bird         NaN   calling             NaN
4     5        fifth       ring         NaN    golden             NaN
5     6        sixth      goose    a-laying       NaN             NaN
6     7      seventh       swan  a-swimming       NaN             NaN
7     8       eighth       maid   a-milking       NaN             NaN
8     9        ninth       lady     dancing       NaN             NaN
9    10        tenth       lord   a-leaping       NaN             NaN
10   11     eleventh      piper      piping       NaN             NaN
11   12      twelfth    drummer    drumming       NaN             NaN


# Function 1: pluralize_gift()

In [4]:
print(xmas['Gift.Item'])

0     partridge
1          dove
2           hen
3          bird
4          ring
5         goose
6          swan
7          maid
8          lady
9          lord
10        piper
11      drummer
Name: Gift.Item, dtype: object


In [5]:
#| code-fold: false

def pluralize_gift(gift = None):
        """
        Returns plural of a noun

        Parameters
        ----------
        gift: str
        A noun
        
        Return
        ------
        str
        Plural version
        """
        if gift is None:
                exit("Add a value to the function.")
        if not gift:
                exit("Don't leave string empty")
        if not isinstance(gift, str):
                exit("Input to pluralize_gift must be a string.")

        if 'oo' in gift:
                gift = gift.replace("oo", "ee")
        elif gift[-1] == 'y':
                gift = gift.replace("y", "ies")
        else:
                gift = gift + "s"

        return gift

In [6]:
# Testing
# Should work
print(pluralize_gift("goose"))
print(pluralize_gift("baby"))
print(pluralize_gift("dog"))
print(pluralize_gift(""))
print(pluralize_gift())



geese
babies
dogs


SystemExit: Don't leave string empty

In [7]:
# Function not vectorized
# pluralize_gift(xmas['Gift.Item'])
for x in xmas['Gift.Item']:
        print(pluralize_gift(x))


partridges
doves
hens
birds
rings
geese
swans
maids
ladies
lords
pipers
drummers


# Function 2: make_phrase()

In [8]:
xmas.dtypes

Day              int64
Day.in.Words    object
Gift.Item       object
Verb            object
Adjective       object
Location        object
dtype: object

In [9]:
#| code-fold: false

# Create a cardinal word column from Day (int) → "one", "two", ..., "twelve"
num_to_word = {
        1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six",
        7:"seven", 8:"eight", 9:"nine", 10:"ten", 11:"eleven", 12:"twelve"
}
xmas["Num.Word"] = xmas["Day"].map(num_to_word)  # use this instead of Day.in.Words

In [10]:
#| code-fold: false

def make_phrase(num, num_word, item, verb, adjective, location):
        """
        Build the gift phrase for each row of the 12 Days of Christmas table.

        Parameters
        ----------
        num : pandas.Series (int)
                Numeric day values (1..12).
        num_word : pandas.Series (str)
                Cardinal number words corresponding to `num` ("one".."twelve").
        item : pandas.Series (str)
                Gift item names (singular). Will be pluralized automatically when day > 1.
        verb : pandas.Series (str)
                Optional action words/phrases for the gift (NAs allowed).
        adjective : pandas.Series (str)
                Optional descriptors for the gift (NAs allowed).
        location : pandas.Series (str)
                Optional location/prepositional phrase (NAs allowed).

        Returns
        -------
        pandas.Series (str)
                A vector of cleaned phrases like "two turtle doves" or "a partridge in a pear tree".
        """
        

        # Step 1: Replace NAs with blank strings (and ensure string dtype)
        verb = verb.fillna("").astype(str)
        adjective = adjective.fillna("").astype(str)
        location = location.fillna("").astype(str)
        item = item.fillna("").astype(str)
        num_word = num_word.fillna("").astype(str)

        # Step 2: Pluralize gift if day > 1 (vectorized, no chained indexing)
        item_plural = item.copy()
        day = num.astype("Int64")
        mask_plural = day > 1
        item_plural.loc[mask_plural] = item.loc[mask_plural].apply(pluralize_gift)

        # Step 3: Does the (singular) gift start with a vowel? (Day 1 needs this)
        # Using item_plural is fine—on day 1 it's identical to item.
        starts_with_vowel = item_plural.str.strip().str[0].str.lower().isin(list("aeiou"))

        # Step 4: Choose determiner: "a"/"an" for day 1; otherwise use num_word
        determiner = num_word.copy()
        day1 = day == 1
        determiner.loc[day1 & starts_with_vowel] = "an"
        determiner.loc[day1 & ~starts_with_vowel] = "a"

        # Step 5: Concatenate pieces and tidy spaces
        phrase = (
                determiner + " " +
                adjective + " " +
                item_plural + " " +
                verb + " " +
                location
        ).str.replace(r"\s+", " ", regex=True).str.strip()

        return phrase

In [11]:
test_df = pd.DataFrame({
        "Day": [1, 2],
        "Num.Word": ["one", "two"],
        "Gift.Item": ["partridge", "turtle dove"],
        "Verb": ["", ""],
        "Adjective": ["", ""],
        "Location": ["in a pear tree", ""]
})
phrases = make_phrase(test_df["Day"], test_df["Num.Word"], test_df["Gift.Item"],
                      test_df["Verb"], test_df["Adjective"], test_df["Location"])
print(phrases)


0    a partridge in a pear tree
1              two turtle doves
dtype: object


In [12]:
#| code-fold: false

xmas["Full.Phrase"] = make_phrase(
        xmas["Day"],
        xmas["Num.Word"],
        xmas["Gift.Item"],
        xmas["Verb"],
        xmas["Adjective"],
        xmas["Location"]
)

print(xmas["Full.Phrase"])

0     a partridge in a pear tree
1               two turtle doves
2              three french hens
3             four calling birds
4              five golden rings
5             six geese a-laying
6         seven swans a-swimming
7          eight maids a-milking
8            nine ladies dancing
9            ten lords a-leaping
10          eleven pipers piping
11      twelve drummers drumming
Name: Full.Phrase, dtype: object


# Function 3: sing_day()

In [ ]:
#| code-fold: false

def sing_day(dataset, num, phrase_col):
        """
        Construct the verse for a specific day of Christmas.

        Parameters
        ----------
        dataset : pandas.DataFrame
                Table containing at least a "Day" column and `phrase_col` with gift phrases.
        num : int
                Day number to sing (1..12).
        phrase_col : str
                Name of the column in `dataset` that holds the phrase text (e.g., "Full.Phrase").

        Returns
        -------
        str
                A multi-line string containing the verse for the requested day,
                including the intro line and properly punctuated gift lines.
        
        """
        ordinals = {
                1: "first", 2: "second", 3: "third", 4: "fourth", 5: "fifth", 6: "sixth",
                7: "seventh", 8: "eighth", 9: "ninth", 10: "tenth", 11: "eleventh", 12: "twelfth"
        }

        if phrase_col not in dataset.columns:
                exit(f"Column '{phrase_col}' not found in dataset.")

        # Step 1: Intro line
        num_word = ordinals.get(num, str(num))
        intro = f"On the {num_word} day of Christmas, my true love gave to me:"

        # Step 2: Gift lines (reverse order)
        gifts = ""
        for i in range(num, 0, -1):
                gift_line = dataset.loc[dataset["Day"] == i, phrase_col].values[0]

                if i == 1 and num != 1:
                # last line, after other gifts
                        gifts += "and " + gift_line + "."
                elif i == 1:
                # only gift (day 1)
                        gifts += gift_line + "."
                else:
                # any middle or top line
                        gifts += gift_line + ",\n"

        # Step 3: Combine
        song = intro + "\n" + gifts
        return song


In [14]:
print(sing_day(xmas, 2, "Full.Phrase"))

On the second day of Christmas, my true love gave to me:
two turtle doves,
and a partridge in a pear tree.


In [15]:
print(sing_day(xmas, 3, "Full.Phrase"))


On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


# Use Your Functions!

In [16]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [17]:
print(xmas2.columns)

Index(['Day', 'Day.in.Words', 'Gift.Item', 'Verb', 'Adjective', 'Location'], dtype='object')


In [18]:
print(xmas2)

    Day Day.in.Words Gift.Item         Verb Adjective       Location
0     1        first     email          NaN       NaN  from Cal Poly
1     2       second     point          NaN      meal            NaN
2     3        third       pen          NaN      lost            NaN
3     4       fourth    review          NaN    course            NaN
4     5        fifth      exam          NaN  practice            NaN
5     6        sixth    grader      grading       NaN            NaN
6     7      seventh    senior    stressing       NaN            NaN
7     8       eighth       mom    a-calling       NaN            NaN
8     9        ninth     party      bumping       NaN            NaN
9    10        tenth      load   of laundry       NaN            NaN
10   11     eleventh    friend  goodbye-ing       NaN            NaN
11   12      twelfth      hour     sleeping       NaN            NaN


In [19]:
#| code-fold: false

# Map number words
num_to_word = {
        1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six",
        7:"seven", 8:"eight", 9:"nine", 10:"ten", 11:"eleven", 12:"twelve"
}
xmas2["Num.Word"] = xmas2["Day"].map(num_to_word)

# Make the full phrase for each row
xmas2["Full.Phrase"] = make_phrase(
        xmas2["Day"], xmas2["Num.Word"],
        xmas2["Gift.Item"], xmas2["Verb"],
        xmas2["Adjective"], xmas2["Location"]
)


In [20]:
print(xmas2["Full.Phrase"] )

0         an email from Cal Poly
1                two meal points
2                three lost pens
3            four course reviews
4            five practice exams
5            six graders grading
6        seven seniors stressing
7           eight moms a-calling
8           nine parties bumping
9           ten loads of laundry
10    eleven friends goodbye-ing
11         twelve hours sleeping
Name: Full.Phrase, dtype: object


In [21]:
#| code-fold: false

def sing_full_song(dataset, phrase_col):
        """
        Sing (print) all verses for the 12 Days of Christmas.

                Parameters
                ----------
                dataset : pandas.DataFrame
                Table containing "Day" (1..12) and a column with gift phrases.
                phrase_col : str
                Name of the column in `dataset` that holds the phrase text
                (e.g., "Full.Phrase" created by `make_phrase`).

                Returns
                -------
                None
                Prints the 12 verses separated by blank lines.
        """
        verses = dataset["Day"].map(lambda d: sing_day(dataset, d, phrase_col))
        print("\n\n".join(verses))


In [22]:
# Original Christmas song
print("The 12 Days of Christmas\n")
sing_full_song(xmas, "Full.Phrase")


The 12 Days of Christmas

On the first day of Christmas, my true love gave to me:
a partridge in a pear tree.

On the second day of Christmas, my true love gave to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love gave to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love gave to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love gave to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love gave to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens

In [23]:
print("The Surprise Song\n")
sing_full_song(xmas2, "Full.Phrase")

The Surprise Song

On the first day of Christmas, my true love gave to me:
an email from Cal Poly.

On the second day of Christmas, my true love gave to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love gave to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love gave to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love gave to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love gave to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love gave to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email f